In [5]:
#impyla demo

from impala.dbapi import connect
conn = connect(host='localhost', port=21050)
cur = conn.cursor()

In [6]:
cur.execute('SHOW TABLES')
cur.fetchall()


[('german',),
 ('german_parquet',),
 ('hbaseplaces',),
 ('jobloss0708',),
 ('mydelta',),
 ('nyse_elt',),
 ('nysetrades',),
 ('places',),
 ('s07',),
 ('s07snappyparquet',),
 ('sample_07',),
 ('sample_08',)]

In [7]:
cur.execute('DESCRIBE german_parquet')
cur.fetchall()


[('cred', 'int', '0  response   - Creditability'),
 ('acct_bal', 'int', '1  cat(1,4)   - Account Balance'),
 ('dur_cred', 'int', '2  cont       - Duration of Credit (month)'),
 ('pay_stat', 'int', '3  cat(0,4)   - Payment Status of Previous Credit'),
 ('purpose', 'int', '4  cat(0,10)  - Purpose'),
 ('cred_amt', 'int', '5  cont       - Credit Amount'),
 ('value', 'int', '6  cat(1,5)   - Value Savings/Stocks'),
 ('len_emp', 'int', '7  cat(1,5)   - Length of current employment'),
 ('install_pc', 'int', '8  cat(1,4)   - Instalment per cent'),
 ('sex_married', 'int', '9  cat(1,4)   - Sex & Marital Status'),
 ('guarantors', 'int', '10 cat(1,3)   - Guarantors'),
 ('dur_addr', 'int', '11 cat(1,4)   - Duration in Current address'),
 ('max_val', 'int', '12 cat(1,4)   - Most valuable available asset'),
 ('age', 'int', '13 cont - Age (years)'),
 ('concurr', 'int', '14 cat(1,3)  - Concurrent Credits'),
 ('typ_aprtmnt', 'int', '15 cat(1,3)  - Type of apartment'),
 ('no_creds', 'int', '16 cat(1,4)  -

In [8]:
cur.execute('SELECT cred, age, value FROM german_parquet LIMIT 20')
cur.fetchall()

[(1, 21, 1),
 (1, 36, 1),
 (1, 23, 2),
 (1, 39, 1),
 (1, 38, 1),
 (1, 48, 1),
 (1, 39, 1),
 (1, 40, 1),
 (1, 65, 1),
 (1, 23, 3),
 (1, 36, 1),
 (1, 24, 2),
 (1, 31, 1),
 (1, 31, 2),
 (1, 23, 5),
 (1, 44, 3),
 (1, 40, 1),
 (1, 25, 3),
 (1, 36, 1),
 (1, 39, 1)]

In [9]:
# However, we want the ability to read WITH metadata
from impala.util import as_pandas
cur.execute('SELECT cred, age, value FROM german_parquet LIMIT 20')
dat = as_pandas(cur)

In [10]:
dat


,cred,age,value
0,1,21,1
1,1,36,1
2,1,23,2
3,1,39,1
4,1,38,1
5,1,48,1
6,1,39,1
7,1,40,1
8,1,65,1
9,1,23,3


In [11]:
# Oh yeah, it also has proper type conversion (meaning, not read as string):
dat.dtypes

cred     int64
age      int64
value    int64
dtype: object

In [12]:
# From here, we can use familiar pandas syntax for data transforms, ie, column select
dat[['cred','value']]

,cred,value
0,1,1
1,1,1
2,1,2
3,1,1
4,1,1
5,1,1
6,1,1
7,1,1
8,1,1
9,1,3


In [13]:
# Be careful doing this on very large results... 
# Here is an example of the power of pandas; the 'Patrick' query
cur.execute('SELECT * FROM german_parquet')
dat = as_pandas(cur)
dat.groupby(['cred']).agg(['nunique','sum'])

acct_bal       dur_cred        pay_stat       purpose       cred_amt  \
      nunique   sum  nunique    sum  nunique   sum nunique   sum  nunique   
cred                                                                        
0           4   571       25   7458        5   650      10   871      294   
1           4  2006       31  13445        5  1895      10  1957      657   

              ...  no_creds      occupation        no_dep      telephone       \
          sum ...   nunique  sum    nunique   sum nunique  sum   nunique  sum   
cred          ...                                                               
0     1181438 ...         4  410          4   881       2  346         2  413   
1     2089810 ...         4  997          4  2023       2  809         2  991   

     foreign_wkr       
         nunique  sum  
cred                   
0              2  304  
1              2  733  

[2 rows x 40 columns]